In [1]:
import selenium.webdriver
from selenium.webdriver import Chrome

from tqdm import tqdm

import csv

In [2]:
gecko = selenium.webdriver.Chrome(executable_path='/home/ksenia/mlcourse_open-master/Yandex/chromedriver')

In [3]:
url = 'https://export.yandex.ru/last/last20x.xml'

In [4]:
search_queries = list()

In [5]:
while len(search_queries) < 10000:
    gecko.get(url)
    list_search_queries = gecko.find_elements_by_class_name('text')
    for query in list_search_queries:
        if len(query.text) != 0 and query.text not in search_queries:
            search_queries.append(query.text)

In [7]:
search_queries = search_queries[:10000]

In [8]:
csvfile = 'search_queries.csv'

with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in search_queries:
        writer.writerow([val]) 